 Desarrollo de la extracción, transformación y carga del conjunto de datos proporcionados  (ETL) de los 3 archivos .json 

       ETL en steam_games

In [71]:
#Importación de librerías
import pandas as pd
import ast
import json
import warnings
warnings.filterwarnings("ignore")

Cargamos el archivo formato .json y lo hacemos un DataFrame

In [72]:
steam_games = 'data/output_steam_games.json'

# Se lee cada línea del dataset
filas_games = []
with open(steam_games) as f:
    for line in f.readlines():
        data = json.loads(line)
        filas_games.append(data)

# Se convierte en dataframe
df_games = pd.DataFrame(filas_games)
df_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,user_id,steam_id,items,items_count
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76561197970982479,76561197970982479,"[{'item_id': '10', 'item_name': 'Counter-Strik...",277.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,js41637,76561198035864385,"[{'item_id': '10', 'item_name': 'Counter-Strik...",888.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,evcentric,76561198007712555,"[{'item_id': '1200', 'item_name': 'Red Orchest...",137.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Riot-Punch,76561197963445855,"[{'item_id': '10', 'item_name': 'Counter-Strik...",328.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,doctr,76561198002099482,"[{'item_id': '300', 'item_name': 'Day of Defea...",541.0


In [73]:
df_games.columns

Index(['publisher', 'genres', 'app_name', 'title', 'url', 'release_date',
       'tags', 'reviews_url', 'discount_price', 'specs', 'price',
       'early_access', 'id', 'metascore', 'developer', 'user_id', 'steam_id',
       'items', 'items_count'],
      dtype='object')

Podemos ver que hay columnas repetidas que estan también en la tabla de users_items por lo que procedemos a borrarlas y también a las demás que no utilizaremos

In [74]:
# Lista de columnas a eliminar
columnas_a_eliminar = ['user_id', 'steam_id', 'items', 'items_count','url','reviews_url','tags','discount_price','specs','metascore','publisher','app_name','early_access','developer']

# Utilizamos el método drop para eliminar las columnas
df_games = df_games.drop(columnas_a_eliminar, axis=1)

In [75]:
#Comprobamos que se hayan eliminado
df_games.columns

Index(['genres', 'title', 'release_date', 'price', 'id'], dtype='object')

Limpiamos los valores nulos

In [76]:
df_games.dropna(inplace=True) #drop valores faltantes

In [77]:
print(df_games.isnull().sum())

genres          0
title           0
release_date    0
price           0
id              0
dtype: int64


In [78]:
# Reemplazamos los valores no válidos por NaN
df_games['release_date'] = pd.to_datetime(df_games['release_date'], errors='coerce')
# Filtramos el DataFrame para obtener los registros válidos
df_filtered = df_games.dropna(subset=['release_date'])
# Convertimos la columna "release_date" a tipo datetime
df_filtered['release_date'] = pd.to_datetime(df_filtered['release_date'])
# Rellenamos los valores faltantes en las columna'release_date' con la moda
columns_fill_mode = ['release_date']
for col in columns_fill_mode:
    df_games[col].fillna(df_games[col].mode()[0], inplace=True)
# Modelado y eliminación de la columna 'release_date'
df_games['year'] = df_games['release_date'].dt.year.astype('Int64')
df_games.drop('release_date', axis=1, inplace=True)

In [79]:
df_games.head()

,genres,title,price,id,year
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,4.99,761140,2018
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,Free To Play,643980,2018
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Free to Play,670290,2017
88313,"[Action, Adventure, Casual]",弹炸人2222,0.99,767400,2017
88315,"[Action, Adventure, Simulation]",Battle Royale Trainer,3.99,772540,2018


In [80]:
df_games.shape

(27617, 5)

In [81]:
df_games.to_csv('data/df_games_clean.csv') #Lo guardo en un csv 

          ETL en users_items

In [82]:
#lista vacía llamada rows. En ella se almacenaran los diccionarios que se obtendrán de las líneas del archivo JSON
rows = []
with open('data/australian_users_items.json', 'r', encoding='UTF-8') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))

In [83]:
user_items = pd.DataFrame(rows)

In [84]:
user_items

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


In [85]:
#Estadísticas descriptivas para todas las columnas numéricas 
user_items.describe()

,items_count
count,88310.000000
mean,58.353629
std,122.312095
min,0.000000
25%,3.000000
50%,26.000000
75%,73.000000
max,7762.000000


Limpieza de valores nulos

In [86]:
user_items = user_items.dropna(thresh=3)
print(user_items.shape)
user_items.head(3)

(88310, 5)


,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."


In [87]:
user_items.isnull().sum()

user_id        0
items_count    0
steam_id       0
user_url       0
items          0
dtype: int64

In [88]:
#Información detallada, incluyendo la cantidad de filas y columnas, el tipo de datos en cada columna y la cantidad de valores no nulos
user_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


La columna 'items' se encuentra anidada, es decir, es una lista de diccionarios. Se procede a generar una normalización para extraer los datos y generar nuevas columnas

In [89]:
# aplico la funcion lambda para desanidar las keys que busco con sus values
user_items['playtime_forever'] = user_items['items'].apply(lambda x: x[0].get('playtime_forever') if len(x) > 0 else 0)
user_items['playtime_2weeks'] = user_items['items'].apply(lambda x: x[0].get('playtime_2weeks') if len(x) > 0 else 0)
user_items['id'] = user_items['items'].apply(lambda x: x[0].get('item_id') if len(x) > 0 else 0)
user_items['playtime_forever'] = user_items['playtime_forever'].astype(int)
user_items['playtime_2weeks'] = user_items['playtime_2weeks'].astype(int)
user_items['item_name'] = user_items['items'].apply(lambda x: x[0].get('item_name') if len(x) > 0 else '')

In [90]:
user_items

,user_id,items_count,steam_id,user_url,items,playtime_forever,playtime_2weeks,id,item_name
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik...",6,0,10,Counter-Strike
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik...",0,0,10,Counter-Strike
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest...",923,0,1200,Red Orchestra: Ostfront 41-45
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik...",0,0,10,Counter-Strike
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea...",1131,0,300,Day of Defeat: Source
...,...,...,...,...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla...",0,0,413850,CS:GO Player Profiles
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania...",0,0,11020,TrackMania Nations Forever
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[],0,0,0,
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'...",677,677,304930,Unturned


In [91]:
#Eliminamos las columnas que no utilizaremos
user_items = user_items.drop(['user_url', 'playtime_2weeks','items','item_name'], axis=1)

In [92]:
user_items.head()

,user_id,items_count,steam_id,playtime_forever,id
0,76561197970982479,277,76561197970982479,6,10
1,js41637,888,76561198035864385,0,10
2,evcentric,137,76561198007712555,923,1200
3,Riot-Punch,328,76561197963445855,0,10
4,doctr,541,76561198002099482,1131,300


In [93]:
user_items.shape

(88310, 5)

In [94]:

user_items.to_csv('data/user_items_clean.csv') #Lo guardo en un csv 

           ETL en user_reviews

In [95]:
rows = []
with open('data/australian_user_reviews.json', 'r', encoding='UTF-8') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))

In [96]:
user_reviews = pd.DataFrame(rows)

In [97]:
user_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


Limpiamos los valores nulos

In [98]:
user_reviews = user_reviews.dropna(thresh=3)
print(user_reviews.shape)
user_reviews.head(3)

(25799, 3)


,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."


Reviews tiene una lista de diccionarios, por lo que debemos desanidar la data que esta anidada 

In [99]:
df_reviewsOpen= user_reviews.explode('reviews') #creo un df con el archivo review y accedo a la columna que me interesa desanidar.
user_reviews_complet = pd.concat([df_reviewsOpen.drop(['reviews'], axis=1), df_reviewsOpen['reviews'].apply(pd.Series)], axis=1) #con esta expresion  creo un nuevo df donde esta la info desanidada en columnas

In [100]:
user_reviews_complet.drop(0, axis=1, inplace=True)

In [101]:
# se crea la columna year
user_reviews_complet['posted year'] = user_reviews_complet['posted'].str.extract(r'(\d{4})')

#Se elimina la columna posted
user_reviews_complet.drop('posted' , axis = 1, inplace = True)

In [102]:
user_reviews_complet

,user_id,user_url,funny,last_edited,item_id,helpful,recommend,review,posted year
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,,22200,No ratings yet,True,It's unique and worth a playthrough.,2011
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,2011
1,js41637,http://steamcommunity.com/id/js41637,,,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2014
1,js41637,http://steamcommunity.com/id/js41637,,,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,2013
...,...,...,...,...,...,...,...,...,...
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,,70,No ratings yet,True,a must have classic from steam definitely wort...,NaN
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,,,362890,No ratings yet,True,this game is a perfect remake of the original ...,NaN
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1 person found this review funny,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,NaN
25798,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,,,730,No ratings yet,True,:D,NaN


In [103]:
#Eliminamos valores nulos
user_reviews_complet.dropna(subset=['posted year'], inplace=True)

Feature Engineering: En el dataset user_reviews se incluyen reseñas de juegos hechos por distintos usuarios. Debes crear la columna 'sentiment_analysis' aplicando análisis de sentimiento con NLP con la siguiente escala: debe tomar el valor '0' si es malo, '1' si es neutral y '2' si es positivo. Esta nueva columna debe reemplazar la de user_reviews.review para facilitar el trabajo de los modelos de machine learning y el análisis de datos. De no ser posible este análisis por estar ausente la reseña escrita, debe tomar el valor de 1.

In [104]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [105]:
#Creamos una instancia del analizador de sentimiento:
sia = SentimentIntensityAnalyzer()

In [106]:
#Iteramos sobre cada reseña en el dataset y realizamos el análisis de sentimiento:
for index, row in user_reviews_complet.iterrows():
    review = row['review']
    if pd.isnull(review):
        sentiment = 1
    else:
        sentiment_scores = sia.polarity_scores(review)
        compound_score = sentiment_scores['compound']
        
        if compound_score >= 0.05:
            sentiment = 2  # Positivo
        elif compound_score <= -0.05:
            sentiment = 0  # Negativo
        else:
            sentiment = 1  # Neutral
    
    user_reviews_complet.at[index, 'sentiment_analysis'] = sentiment

In [107]:
user_reviews_complet.head()

,user_id,user_url,funny,last_edited,item_id,helpful,recommend,review,posted year,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011,2.0
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,,22200,No ratings yet,True,It's unique and worth a playthrough.,2011,2.0
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,2011,2.0
1,js41637,http://steamcommunity.com/id/js41637,,,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2014,2.0
1,js41637,http://steamcommunity.com/id/js41637,,,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,2013,2.0


In [108]:
#Observamos la cantidad de resultados
sentiment_counts = user_reviews_complet['sentiment_analysis'].value_counts()
sentiment_counts

sentiment_analysis
2.0    32147
1.0     9947
0.0     7092
Name: count, dtype: int64

In [109]:
#Eliminamos las columnas que no nos sirven para el analisis
df_reviews = user_reviews_complet.drop(columns=['review','funny','last_edited', 'helpful', 'user_url'])
df_reviews.columns

Index(['user_id', 'item_id', 'recommend', 'posted year', 'sentiment_analysis'], dtype='object')

In [110]:
df_reviews

,user_id,item_id,recommend,posted year,sentiment_analysis
0,76561197970982479,1250,True,2011,2.0
0,76561197970982479,22200,True,2011,2.0
0,76561197970982479,43110,True,2011,2.0
1,js41637,251610,True,2014,2.0
1,js41637,227300,True,2013,2.0
...,...,...,...,...,...
25764,wayfeng,730,True,2015,1.0
25765,76561198251004808,253980,True,2015,2.0
25769,72947282842,730,True,2015,0.0
25771,ApxLGhost,730,True,2015,2.0


In [111]:
df_rewiews.to_csv('data/user_reviews_clean.csv', index=False) 